
# Bird Species Classification - Deep Learning Project
**Akbank Derin Öğrenme Bootcamp - Bird Species Dataset**

Bu çalışmada Kaggle üzerindeki *Bird Species Classification* veri seti ile farklı kuş türlerini sınıflandırmayı deneyeceğiz. Adım adım ilerleyip hem kod hem de açıklamalar ekledim, böylece süreci takip etmek kolay olacak.

**İçerik (kısa):**
1. Proje amacı ve veri tanımı
2. Gerekli paketlerin kurulumu ve ortam ayarları
3. Veri indirme (Kaggle) ve klasör yapısı
4. Keşifsel Veri Analizi (EDA) - örnek görselleştirmeler
5. Veri ön işleme ve Data Augmentation
6. Model mimarisi (Transfer Learning: EfficientNetB0) + alternatif basit CNN
7. Eğitim, doğrulama, eğitim grafikleri
8. Değerlendirme: Confusion Matrix, Classification Report
9. Görselleştirme: Grad-CAM (model attention map)
10. Basit Hiperparametre arama (örnek Keras Tuner kullanımı)
11. Sonuçlar ve README için özet



## 1) Proje Tanıtımı

Bu projede amaç, kuş türlerini doğru şekilde sınıflandırabilecek bir derin öğrenme modeli geliştirmektir.  
Kullanılacak veri seti Kaggle'da yer alan **Bird Species Classification** veri setidir.  

Bu bölümde kısaca şunlardan bahsediyoruz:
- Projenin hedefi
- Kullanılacak veri setinin temel özellikleri
- Çalışmada izlenecek yol haritası

Sonraki adımlarda ortam hazırlığı, veri ön işleme, model kurulumu, eğitim ve değerlendirme aşamaları adım adım gösterilecektir.



## 2) Ortam Hazırlığı ve Gereksinimler

Aşağıdaki hücreler ortam kurulumunu (gerekirse) yapar. Kaggle'dan veri indirmeniz için `kaggle` paketi ve API token'ına ihtiyaç vardır. Eğer Kaggle kullanmak istemezseniz, veri setini kendiniz indirip notebook çalışma dizinine yükleyebilirsiniz.

> **Not:** Bu notebook Kaggle üzerinde çalışacak şekilde tasarlanmıştır. Localde çalıştırırken Kaggle API token (`kaggle.json`) veya manuel indirme yoluyla veri yüklenmelidir.


In [ ]:

# Gerekli kütüphaneleri yüklüyoruz (Kaggle ortamında çoğu zaten mevcut)
# Eğer Kaggle ortamında çalışıyorsanız bu hücreyi atlayabilirsiniz.
!pip install -q kaggle tensorflow matplotlib scikit-learn pandas seaborn opencv-python-headless tensorflow-addons
!pip install -q -U keras-tuner



## 3) Veri Setini İndirme

Aşağıdaki adımlar Kaggle API ile veri indirme içindir:
1. Kaggle hesabınızda **Account > API** kısmından `kaggle.json` dosyasını indirin.
2. Notebook ortamınıza `kaggle.json` dosyasını yükleyin (Kaggle'da dosyayı doğrudan yüklediyseniz bu adımı atlayın).
3. Aşağıdaki hücreyi çalıştırarak veri setini indirin.

**Dataset:** https://www.kaggle.com/datasets/akash2907/bird-species-classification


In [ ]:

# Kaggle API üzerinden veri indirme (çalışması için kaggle.json yüklenmiş olmalı)
import os
if not os.path.exists('kaggle.json'):
    print('kaggle.json bulunamadı - eğer Kaggle üzerinde çalışıyorsanız bu uyarıyı göz ardı edin.')
else:
    os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
    !kaggle datasets download -d akash2907/bird-species-classification -p ./ --unzip
    print('İndirme tamamlandı.')



## 4) Veri Yapısı ve Keşifsel Analiz (EDA)

Veri setini indirdikten sonra klasör yapısını kontrol edin. Aşağıda örnek olarak `train/` içinde sınıf klasörleri olduğu varsayılıyor. Hedefimiz resimleri `ImageDataGenerator` veya `tf.keras.utils.image_dataset_from_directory` ile yüklemektir.

Ayrıca sınıf dağılımını görselleştirerek dengesizlik olup olmadığını kontrol edin.


In [ ]:

# Klasör yapısını kontrol edelim
import os, pathlib
base_dir = './train'  # eğer farklı ise burayı güncelleyin
if os.path.exists(base_dir):
    classes = [d.name for d in pathlib.Path(base_dir).iterdir() if d.is_dir()]
    print(f'{len(classes)} sınıf bulundu. İlk 10 sınıf örneği:', classes[:10])
else:
    print(f'{base_dir} bulunamadı. Lütfen veri dizinini kontrol edin.')


In [ ]:

# Sınıf dağılımını görselleştiriyoruz
import matplotlib.pyplot as plt, seaborn as sns, pandas as pd
if os.path.exists(base_dir):
    counts = {c: len(list(pathlib.Path(base_dir).joinpath(c).glob('*.jpg'))) for c in classes}
    df_counts = pd.DataFrame.from_dict(counts, orient='index', columns=['count']).sort_values('count', ascending=False)
    display(df_counts.head(20))
    plt.figure(figsize=(10,6))
    sns.barplot(x=df_counts.index[:20], y='count', data=df_counts.reset_index().rename(columns={'index':'class'}))
    plt.xticks(rotation=90)
    plt.title('Top 20 sınıf örnek sayısı')
    plt.show()
else:
    print('Veri bulunamadığı için EDA atlandı.')



## 5) Veri Ön İşleme ve Data Augmentation

Bu bölümde `tf.keras.preprocessing.image.ImageDataGenerator` veya `tf.data` API kullanacağız. Örnek olarak `image_dataset_from_directory` (tf.data tabanlı) gösterilecektir. Ayrıca eğitim sırasında augmentasyon uygulanacaktır.


In [ ]:

import tensorflow as tf
IMAGE_SIZE = (224,224)
BATCH_SIZE = 32
AUTOTUNE = tf.data.AUTOTUNE

def get_datasets(train_dir, val_dir, image_size=IMAGE_SIZE, batch_size=BATCH_SIZE):
    train_ds = tf.keras.preprocessing.image_dataset_from_directory(
        train_dir,
        label_mode='categorical',
        image_size=image_size,
        batch_size=batch_size,
        shuffle=True
    )
    val_ds = tf.keras.preprocessing.image_dataset_from_directory(
        val_dir,
        label_mode='categorical',
        image_size=image_size,
        batch_size=batch_size,
        shuffle=False
    )
    class_names = train_ds.class_names
    return train_ds, val_ds, class_names

print('Dataset pipeline hazır - veri dosyalarınız mevcutsa train/ ve val/ yollarını vererek kullanabilirsiniz.')



### Data Augmentation (tf.keras.layers üzerinden)
Burada `tf.keras.Sequential` içinde augmentasyon layer'ları kullanacağız. Bu GPU üzerinde verimli çalışır.


In [ ]:

from tensorflow.keras import layers
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip('horizontal'),
    layers.RandomRotation(0.12),
    layers.RandomZoom(0.1),
    layers.RandomContrast(0.1),
], name='data_augmentation')

# Augmentation örneklerini görselleştirelim
def visualize_augmentation(dataset, n=3):
    import matplotlib.pyplot as plt
    for images, labels in dataset.take(1):
        plt.figure(figsize=(12,4))
        for i in range(n):
            ax = plt.subplot(1,n,i+1)
            aug = data_augmentation(tf.expand_dims(images[i], 0))
            plt.imshow(aug[0].numpy().astype('uint8'))
            plt.axis('off')
        plt.show()

print('Augmentation layer hazır.')



## 5) Model - Transfer Learning örneği (EfficientNetB0)

Transfer learning kullanarak önceden eğitilmiş bir ağdan faydalanacağız. Bu örnek `tf.keras.applications.EfficientNetB0` kullanır. Son katmanlarda dropout ve dense katmanları ile sınıflandırma yapacağız.


In [ ]:

from tensorflow.keras import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout, Input
from tensorflow.keras.applications import EfficientNetB0
from tensorflow.keras.optimizers import Adam

def build_model(num_classes, image_size=IMAGE_SIZE, lr=1e-4, freeze_base=True):
    inputs = Input(shape=(image_size[0], image_size[1], 3))
    x = data_augmentation(inputs)
    base = EfficientNetB0(include_top=False, input_tensor=x, weights='imagenet')
    if freeze_base:
        base.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(0.3)(x)
    outputs = Dense(num_classes, activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(lr), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

print('Model oluşturma fonksiyonu hazır.')



## 7) Modelin Eğitilmesi

Aşağıdaki hücre, modelin nasıl eğitileceğini gösterir. Callbacks: ModelCheckpoint, ReduceLROnPlateau, EarlyStopping kullanacağız.


In [ ]:

from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
import os

# Eğitim ve doğrulama veri dizinlerini buraya girmeliyiz
train_dir = './train'
val_dir = './valid'  # ya da ayrı val dizini yoksa farklı bir yol kullanın

# Eğer veriler hazırsa dataset pipeline'ını çalıştıralım
if os.path.exists(train_dir) and os.path.exists(val_dir):
    train_ds, val_ds, class_names = get_datasets(train_dir, val_dir)
    num_classes = len(class_names)
    model = build_model(num_classes=num_classes, freeze_base=True)
    checkpoint = ModelCheckpoint('best_model.h5', monitor='val_accuracy', save_best_only=True, mode='max')
    early = EarlyStopping(monitor='val_loss', patience=6, restore_best_weights=True)
    reduce_lr = ReduceLROnPlateau(monitor='val_loss', factor=0.5, patience=3, min_lr=1e-7)
    history = model.fit(train_ds, validation_data=val_ds, epochs=20, callbacks=[checkpoint, early, reduce_lr])
else:
    print('Eğitim atlandı - lütfen train_dir ve val_dir yollarını kontrol edin.')



## 8) Modelin Değerlendirilmesi

Modelin eğitim/validasyon loss & accuracy grafikleri ile değerlendirmesini yapın. Ayrıca test setiniz varsa test üzerinde performansı hesaplayın.


In [ ]:

import matplotlib.pyplot as plt
def plot_history(history):
    plt.figure(figsize=(12,4))
    plt.subplot(1,2,1)
    plt.plot(history.history['loss'], label='train_loss')
    plt.plot(history.history['val_loss'], label='val_loss')
    plt.legend(); plt.title('Loss')
    plt.subplot(1,2,2)
    plt.plot(history.history['accuracy'], label='train_acc')
    plt.plot(history.history['val_accuracy'], label='val_acc')
    plt.legend(); plt.title('Accuracy')
    plt.show()

if 'history' in globals():
    plot_history(history)
else:
    print('Eğitim sonucu bulunamadı - history değişkeni mevcut değil.')



### Confusion Matrix & Classification Report


In [ ]:

import numpy as np
from sklearn.metrics import classification_report, confusion_matrix
import seaborn as sns

if os.path.exists('best_model.h5') and os.path.exists(val_dir):
    model.load_weights('best_model.h5')
    y_true = []
    y_pred = []
    for images, labels_batch in val_ds:
        preds = model.predict(images)
        y_true.extend(np.argmax(labels_batch.numpy(), axis=1))
        y_pred.extend(np.argmax(preds, axis=1))
    print(classification_report(y_true, y_pred, target_names=class_names))
    cm = confusion_matrix(y_true, y_pred)
    plt.figure(figsize=(10,8))
    sns.heatmap(cm, annot=False, fmt='d')
    plt.title('Confusion Matrix')
    plt.show()
else:
    print('Confusion matrix hesaplaması atlandı. Model veya validation set bulunamadı.')



## 9) Grad-CAM ile Modelin Nereye Baktığını Görselleştirme

Aşağıdaki fonksiyon Grad-CAM hesaplayıp test görüntüleri üzerinde modelin hangi bölgelere dikkat ettiğini gösterir.


In [ ]:

import tensorflow as tf
import numpy as np
import cv2
from tensorflow.keras.models import Model

def get_gradcam_heatmap(model, img_array, last_conv_layer_name):
    grad_model = Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        conv_outputs, predictions = grad_model(img_array)
        pred_index = tf.argmax(predictions[0])
        class_channel = predictions[:, pred_index]
    grads = tape.gradient(class_channel, conv_outputs)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    conv_outputs = conv_outputs[0]
    heatmap = conv_outputs @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / (tf.math.reduce_max(heatmap) + 1e-8)
    return heatmap.numpy()

def display_gradcam(img_path, model, last_conv_layer_name='top_conv'):
    img = tf.keras.preprocessing.image.load_img(img_path, target_size=IMAGE_SIZE)
    img_array = tf.keras.preprocessing.image.img_to_array(img)
    input_arr = np.expand_dims(img_array, axis=0) / 255.0
    heatmap = get_gradcam_heatmap(model, input_arr, last_conv_layer_name)
    heatmap = cv2.resize(heatmap, (img_array.shape[1], img_array.shape[0]))
    heatmap = np.uint8(255 * heatmap)
    heatmap = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    superimposed_img = heatmap * 0.4 + img_array
    plt.figure(figsize=(8,4))
    plt.subplot(1,2,1); plt.imshow(img_array.astype('uint8')); plt.axis('off'); plt.title('Original')
    plt.subplot(1,2,2); plt.imshow(superimposed_img.astype('uint8')); plt.axis('off'); plt.title('Grad-CAM')
    plt.show()

print('Grad-CAM fonksiyonları hazır.')



## 10) Hiperparametre Optimizasyonu (Keras Tuner ile Deneme)

Basit bir Keras Tuner örneği gösterilmektedir. Bu hücre isteğe bağlıdır ve Kaggle ortamında çalıştırmadan önce kaynak ve zaman gereksinimlerini göz önünde bulundurun.


In [ ]:

import kerastuner as kt

def build_model_hp(hp):
    inputs = Input(shape=(IMAGE_SIZE[0], IMAGE_SIZE[1], 3))
    x = data_augmentation(inputs)
    base = EfficientNetB0(include_top=False, input_tensor=x, weights='imagenet')
    base.trainable = False
    x = GlobalAveragePooling2D()(base.output)
    x = Dropout(hp.Float('dropout', 0.2, 0.5, step=0.1))(x)
    units = hp.Choice('units', [128, 256, 512])
    x = Dense(units, activation='relu')(x)
    outputs = Dense(len(class_names), activation='softmax')(x)
    model = Model(inputs, outputs)
    model.compile(optimizer=Adam(hp.Choice('lr', [1e-3, 1e-4, 1e-5])), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

# Örnek tuner (yorum satırı)
# tuner = kt.RandomSearch(build_model_hp, objective='val_accuracy', max_trials=5, directory='tuner_dir', project_name='bird_tuning')
# tuner.search(train_ds, validation_data=val_ds, epochs=5)
print('Keras Tuner örneği hazır (yorum satırı halinde).')


## 11) Genel Değerlendirme ve README İçin Notlar



- **Projenin amacı:** Bird species sınıflandırma.
- **Veri seti:** Kaggle `akash2907/bird-species-classification` - ~9.000 görüntü, 15 sınıf (not: gerçek sayı değişebilir).
- **Yöntem:** Transfer learning (EfficientNetB0), data augmentation, Grad-CAM analizi.
- **Elde edilen sonuçlar:** (Eğitim sonrası en iyi doğruluk, loss, confusion matrix özetleri)
- **Hiperparametre deneyleri:** ( learning rate, dropout, batch size)
- **Çalıştırma talimatları:** Kaggle'da çalıştırmak için `kaggle.json` yüklenmeli, notebook hücreleri sırasıyla çalıştırılmalıdır.
- **Kaggle Notebook linki:** (https://www.kaggle.com/aleynacamlibel)

---

